In [0]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'jsmazorra/JSON API Authorization/'

In [0]:
from google.cloud import bigquery
import time
import pandas as pd
import numpy as np

In [0]:
# Setup client
bq_client = bigquery.Client() 

In [0]:
# GETS SQL QUERY FOR 100 USERS (14,000-14,000) FROM SORTED TABLE OF MOST COMMENTS
# WHERE TOTAL COMMENTS IS SLIGHTLY ABOVE 100

QUERY = '''
        SELECT E.*, C.comments_count
        FROM `bigquery-public-data.hacker_news.full_201510` as E
        JOIN(
            SELECT *
            FROM(
                SELECT *
                FROM(
                    SELECT  `bigquery-public-data.hacker_news.full_201510`.by, COUNT(*) as comments_count
                    FROM `bigquery-public-data.hacker_news.full_201510`
                    GROUP BY `bigquery-public-data.hacker_news.full_201510`.by
                )  
                ORDER BY comments_count DESC
                LIMIT 9500
            )
            ORDER BY comments_count 
            LIMIT 100            
        ) C 
        ON E.by = C.by
        '''

In [10]:
import time
start = time.time()
df = bq_client.query(QUERY).to_dataframe()
end = time.time()
print(end - start)

11.884548664093018


In [11]:
df.shape

(20089, 14)

In [12]:
users = df['by'].unique()
len(users)

100

In [13]:
df.head()

,by,score,time,title,type,url,text,parent,deleted,dead,descendants,id,ranking,comments_count
0,okasaki,NaN,1438624899,None,comment,None,It&#x27;s possible that very few visitors of p...,9998049.0,None,None,NaN,9998345,None,201
1,alecsmart1,NaN,1390616800,None,comment,None,I hate it when blogs don&#x27;t have links to ...,7118342.0,None,None,NaN,7119575,None,200
2,tjbiddle,NaN,1431362729,None,comment,None,Neat! Nice work! Some are definitely criticizi...,9522489.0,None,None,NaN,9526051,None,201
3,b0rsuk,NaN,1375483437,None,comment,None,That&#x27;s a whole world of spying opportunit...,6148347.0,None,None,NaN,6150135,None,200
4,sqs,NaN,1383046380,None,comment,None,Awesome site! It would be great if you added a...,6630831.0,None,None,NaN,6633275,None,201


In [0]:
# Drop unecessary columns
df_drop = df.drop(columns= ['score','title','url','deleted','dead','descendants','ranking'])

# Keep only comments in the dataframe
df_drop = df_drop[df_drop['type']=='comment']

# Drop empty comments 
df_drop = df_drop[(df_drop['text'] != '')&(df_drop['text'] != np.nan)]

# Fix weird text encodings
import html
df_drop['text']=df_drop['text'].apply(str)
df_drop['text'] = df_drop['text'].apply(lambda x: html.unescape(x))

# Remove html tags from string
import re
def remove_html_tags(text):
   clean = re.compile('<.*?>')
   return re.sub(clean, '', text)
df_drop['text'] = df_drop['text'].apply(lambda x: remove_html_tags(x))

# Convert unix time to datetime object with date
from datetime import datetime
df_drop['time']=pd.to_datetime(df_drop['time'],unit='s') # for accuracy secs

In [15]:
# Check only analyzing comments
df_drop['type'].value_counts()

comment    16269
Name: type, dtype: int64

In [16]:
users = df_drop['by'].unique()
print(len(users))

98


In [17]:
df_drop.head()

,by,time,type,text,parent,id,comments_count
0,okasaki,2015-08-03 18:01:39,comment,It's possible that very few visitors of panopt...,9998049.0,9998345,201
1,alecsmart1,2014-01-25 02:26:40,comment,I hate it when blogs don't have links to the m...,7118342.0,7119575,200
2,tjbiddle,2015-05-11 16:45:29,comment,Neat! Nice work! Some are definitely criticizi...,9522489.0,9526051,201
3,b0rsuk,2013-08-02 22:43:57,comment,That's a whole world of spying opportunities. ...,6148347.0,6150135,200
4,sqs,2013-10-29 11:33:00,comment,Awesome site! It would be great if you added a...,6630831.0,6633275,201


In [19]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 2.8MB/s 


In [0]:
# Populate sentiment analysis columns in dataframe
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [0]:
df_drop['neg']=np.zeros(df_drop.shape[0])
df_drop['pos']=np.zeros(df_drop.shape[0])
df_drop['neu']=np.zeros(df_drop.shape[0])

In [22]:
now=time.time()
df_drop['neg'] = df_drop['text'].apply(lambda x: sia.polarity_scores(x)['neg'])
df_drop['pos'] = df_drop['text'].apply(lambda x: sia.polarity_scores(x)['pos'])
df_drop['neu'] = df_drop['text'].apply(lambda x: sia.polarity_scores(x)['neu'])
print('sentiment cal took:',round(time.time()-now,2),'s')

sentiment cal took: 36.26 s


In [24]:
df_drop.head()

,by,time,type,text,parent,id,comments_count,neg,pos,neu
0,okasaki,2015-08-03 18:01:39,comment,It's possible that very few visitors of panopt...,9998049.0,9998345,201,0.000,0.000,1.000
1,alecsmart1,2014-01-25 02:26:40,comment,I hate it when blogs don't have links to the m...,7118342.0,7119575,200,0.133,0.049,0.817
2,tjbiddle,2015-05-11 16:45:29,comment,Neat! Nice work! Some are definitely criticizi...,9522489.0,9526051,201,0.050,0.076,0.873
3,b0rsuk,2013-08-02 22:43:57,comment,That's a whole world of spying opportunities. ...,6148347.0,6150135,200,0.032,0.170,0.798
4,sqs,2013-10-29 11:33:00,comment,Awesome site! It would be great if you added a...,6630831.0,6633275,201,0.000,0.239,0.761


In [25]:
df_final = pd.DataFrame(data=None, columns=df_drop.columns)
df_user_most_pos = pd.DataFrame(data=None, columns=df_drop.columns)
df_user_most_neg = pd.DataFrame(data=None, columns=df_drop.columns)

for user in users:
    # Create a dataframe of only one user
    df_user = df_drop[df_drop['by'] == user]
    
    # Remove pure sentiments (removes pure one-word comments)
    df_user_unpure = df_user[(df_user['neg'] !=1) & (df_user['pos'] !=1)]
    
    # Submit first 100 comments of the unpure sentiments to final dataframe
    df_final = df_final.append(df_user_unpure[:100])
    
    # Submit max unpure neg/pos comment per user into dataframe
    df_temp = df_final[df_final['by'] == user]
    ix_neg = df_temp['neg'].idxmax(axis=0)
    df_user_most_neg = df_user_most_neg.append(df_temp.loc[ix_neg])
    ix_pos = df_temp['pos'].idxmax(axis=0)
    df_user_most_pos = df_user_most_pos.append(df_temp.loc[ix_pos])
    
df_final.shape

(9132, 10)

In [26]:
users = df_drop['by'].unique()
len(users)

98

In [27]:
df_final.head(10)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
0,okasaki,2015-08-03 18:01:39,comment,It's possible that very few visitors of panopt...,9998049.0,9998345,201,0.000,0.000,1.000
70,okasaki,2014-12-25 15:16:55,comment,"No money involved, but you can make prediction...",8796362.0,8796382,201,0.151,0.000,0.849
82,okasaki,2015-08-16 05:56:31,comment,> PDF.js has fewer security vulnerabilities th...,10067897.0,10068133,201,0.037,0.113,0.851
86,okasaki,2015-02-26 19:53:54,comment,">He thought, he judged, as animals can be seen...",9113285.0,9114988,201,0.030,0.177,0.793
307,okasaki,2015-08-17 06:54:33,comment,Why does it need to be user controlled?Someone...,10071444.0,10071633,201,0.000,0.095,0.905
393,okasaki,2014-08-30 18:15:50,comment,"They give it to the NSA, and the NSA shares it...",8246956.0,8247118,201,0.056,0.193,0.750
653,okasaki,2015-05-17 09:04:21,comment,> Employees at the center referred her to the ...,9559134.0,9559296,201,0.154,0.102,0.744
683,okasaki,2014-02-03 17:42:46,comment,Wouldn't it be easier to just gain access to t...,7170623.0,7172175,201,0.107,0.156,0.736
704,okasaki,2014-07-01 15:06:00,comment,Ubuntu ships with AppArmor enabled. Fedora shi...,7971006.0,7971107,201,0.000,0.083,0.917
760,okasaki,2014-01-19 18:38:22,comment,The NSA probably isn't thwarted by TLS. I imag...,7085071.0,7085374,201,0.000,0.067,0.933


In [28]:
# Random comments from the 100 users
df_random100 = df_final.sample(100)
df_random100.head(10)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
10715,bowyakka,2015-10-10 23:09:17,comment,https://www.stickermule.com/marketplace/3442-t...,10366326.0,10367356,201,0.000,0.000,1.000
4725,tjbiddle,2013-09-26 16:18:42,comment,Great job! Looking forward to other similar to...,6450599.0,6451761,201,0.102,0.279,0.619
8275,portLAN,2007-08-24 01:05:46,comment,This is a collection of points PG has made bef...,45698.0,45950,202,0.000,0.183,0.817
5036,collypops,2011-10-02 21:20:35,comment,The Facebook blocker isn't to blame here. It's...,3063172.0,3064055,201,0.000,0.081,0.919
7120,domp,2007-04-02 23:14:06,comment,Once they start giving the artist's a bigger c...,8278.0,8355,201,0.116,0.111,0.773
7983,gradstudent,2011-08-19 02:04:21,comment,Perhaps we have different expectations of what...,2899034.0,2901919,201,0.030,0.039,0.931
2654,dan00,2012-10-20 10:26:20,comment,"""So, on the one hand a certain openness and te...",4674934.0,4676771,201,0.062,0.156,0.782
3677,shadytrees,2009-08-30 00:38:03,comment,> [Writers] ... are busily grafting the sophis...,793052.0,793874,200,0.107,0.091,0.802
5063,downer,2007-11-30 19:00:41,comment,> This is how they want their profiles to look...,84807.0,85079,201,0.153,0.055,0.792
9206,louislouis,2008-06-15 15:53:04,comment,"It's quite an extraordinary achievement, but a...",217889.0,218084,202,0.059,0.075,0.866


In [30]:
# Most negative comments
df_neg100 = df_final.sort_values(['neg'],ascending=False)[:100]
df_neg100.head(20)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
3293,alexkearns,2011-04-24 07:44:02,comment,Stop spamming: http://news.ycombinator.com/sub...,2478465.0,2478472,201,0.841,0.000,0.159
12359,shadytrees,2014-08-31 19:38:17,comment,My bad!,8245728.0,8250164,200,0.791,0.000,0.209
940,yuchi,2013-03-21 13:37:51,comment,Wrong place?,5414864.0,5414898,201,0.756,0.000,0.244
3440,jdunck,2011-07-18 03:39:35,comment,"window.console.log.apply(window.console, argum...",2774920.0,2775026,201,0.730,0.000,0.270
7300,gradstudent,2015-06-12 10:51:26,comment,Yeah. Shitty blogspam posts need to die die die.,9704950.0,9705068,201,0.712,0.102,0.186
3267,yuchi,2012-06-26 17:46:15,comment,"Sorry, but this is a dupe! :\",4163034.0,4163176,201,0.656,0.000,0.344
7537,reddotX,2015-08-04 13:53:17,comment,what github fiasco?,10003348.0,10003370,201,0.623,0.000,0.377
6243,polynomial,2013-08-18 01:21:28,comment,Often wildly wrong.,6229483.0,6231161,200,0.608,0.000,0.392
4639,leoh,2013-01-15 03:06:07,comment,Very unethical and unprofessional of The Atlantic,5057819.0,5058346,200,0.589,0.000,0.411
6012,okasaki,2014-02-24 08:56:12,comment,Painful? 190PPI isn't great but it's far from ...,7289729.0,7289772,201,0.570,0.000,0.430


In [31]:
# Most positive comments
df_pos100 = df_final.sort_values(['pos'],ascending=False)[:100]
df_pos100.head(20)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
12706,domp,2007-04-03 02:28:35,comment,Wow thats pretty amazing.,8396.0,8401,201,0.0,0.915,0.085
5033,maneesh,2009-08-10 06:06:54,comment,That's awesome. Thanks.,752058.0,752074,201,0.0,0.875,0.125
6424,pknight,2013-10-08 01:35:55,comment,"awesome, thank you!",6508090.0,6512467,200,0.0,0.873,0.127
12523,medianama,2010-03-07 19:15:11,comment,Great. Thank You.,1173801.0,1173817,200,0.0,0.868,0.132
2642,nav,2009-06-18 23:32:32,comment,Thanks. Was fun.,664341.0,664342,201,0.0,0.861,0.139
16590,daviday,2012-06-02 17:36:34,comment,Thanks for sharing!,4057945.0,4058591,202,0.0,0.857,0.143
9019,mikeg8,2014-02-12 17:33:44,comment,Interesting perspective. Thanks.,7225297.0,7225758,200,0.0,0.848,0.152
9264,mikeg8,2012-03-24 22:05:40,comment,Interesting point. Thanks.,3750879.0,3750903,200,0.0,0.848,0.152
16610,DavidChouinard,2012-02-16 13:07:44,comment,Care to share?,3598180.0,3598418,201,0.0,0.844,0.156
8063,vidyesh,2010-11-14 07:40:13,comment,Awesome gallery!,1902351.0,1902795,202,0.0,0.815,0.185


In [32]:
# Most single negative comment for each user
df_user_most_neg.head(10)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
6012,okasaki,2014-02-24 08:56:12,comment,Painful? 190PPI isn't great but it's far from ...,7289729.0,7289772,201,0.570,0.000,0.430
1446,alecsmart1,2014-02-20 04:02:57,comment,What was wrong with MJ?,7267960.0,7268784,200,0.437,0.000,0.563
1989,tjbiddle,2015-02-10 17:30:29,comment,I received an email last week - maybe check yo...,9028522.0,9028555,201,0.200,0.000,0.800
4068,b0rsuk,2013-04-23 05:59:18,comment,Don't captchas solve this ? Are spam tools abl...,5590717.0,5593509,200,0.263,0.000,0.737
4603,sqs,2014-03-05 00:29:51,comment,The risk that something like this is what happ...,7343796.0,7344351,201,0.175,0.135,0.690
940,yuchi,2013-03-21 13:37:51,comment,Wrong place?,5414864.0,5414898,201,0.756,0.000,0.244
7371,mtarnovan,2010-05-19 19:37:43,comment,Very silly and naive article.,1361966.0,1362071,201,0.350,0.206,0.444
6553,drjesusphd,2013-09-28 03:25:24,comment,WTF are you talking about?,6460496.0,6460773,202,0.531,0.000,0.469
5596,dan00,2010-05-22 09:35:14,comment,"Oh, I've only commented the title of the post....",1364182.0,1370077,201,0.243,0.162,0.595
5084,pknight,2012-07-26 01:11:52,comment,"No WPEngine hosted sites, or their own domain ...",4286610.0,4293896,200,0.268,0.000,0.732


In [33]:
# Most single positive comment for each user
df_user_most_pos.head(10)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
8234,okasaki,2015-05-27 12:02:12,comment,Rust has much better performance.,9610685.0,9610718,201,0.000,0.420,0.580
12021,alecsmart1,2013-10-25 20:22:51,comment,Thank you,6612849.0,6614314,200,0.000,0.714,0.286
5060,tjbiddle,2013-01-22 05:00:56,comment,Very cool. Great job!,5094466.0,5095623,201,0.000,0.784,0.216
2627,b0rsuk,2012-05-13 20:32:16,comment,I had no idea SNUSP is so whimsical. Brilliant !,3967488.0,3968194,200,0.144,0.399,0.457
2883,sqs,2014-02-19 21:49:32,comment,We use ZenPayroll (at Sourcegraph) and love it...,7264844.0,7266483,201,0.000,0.595,0.405
5026,yuchi,2012-11-07 20:03:24,comment,Thank you for sharing.,4754520.0,4754922,201,0.000,0.726,0.274
1286,mtarnovan,2009-04-01 06:25:10,comment,Sounds like april's fool to me :),541433.0,541465,201,0.234,0.444,0.323
8737,drjesusphd,2015-05-06 18:03:18,comment,Yes they are.,9500397.0,9500571,202,0.000,0.574,0.426
3992,dan00,2010-05-19 11:53:11,comment,Cool stuff!,1360643.0,1360714,201,0.000,0.722,0.278
6424,pknight,2013-10-08 01:35:55,comment,"awesome, thank you!",6508090.0,6512467,200,0.000,0.873,0.127
